# Link Prediction: Amazon Product Co-Purchasing Network

This notebook implements and compares link prediction methods to predict missing links in the Amazon co-purchasing network.

## Objectives:
1. Load train graph and test edges (positive and negative)
2. Implement similarity-based methods (Common Neighbors, Adamic-Adar, Jaccard)
3. Evaluate each similarity method with metrics and visualizations
4. Implement ML-based method (Random Forest) with feature extraction
5. Compare all methods comprehensively
6. Analyze results and provide business insights
7. Save all results


## 1. Import Libraries and Setup

Import all necessary libraries for link prediction, evaluation, and visualization.


In [ ]:
# Standard library imports
import os
import sys
import time
import pickle
from pathlib import Path

# Add src directory to path for imports
if Path.cwd().name == 'notebooks':
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()

sys.path.insert(0, str(project_root / 'src'))
os.chdir(project_root)
print(f"Project root: {project_root}")
print(f"Working directory: {os.getcwd()}")

# Data manipulation
import pandas as pd
import numpy as np

# Network analysis
import networkx as nx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Project modules
from data_loader import load_saved_graph
from link_prediction import (
    common_neighbors_score,
    adamic_adar_score,
    jaccard_coefficient_score,
    calculate_all_similarity_scores,
    predict_links_similarity,
    evaluate_link_prediction
)
from ml_link_prediction import (
    extract_edge_features,
    prepare_training_data,
    train_random_forest,
    predict_links_ml,
    evaluate_ml_model,
    get_feature_importance
)
from link_prediction_evaluation import (
    plot_roc_curves,
    plot_precision_recall_curves,
    plot_confusion_matrices,
    plot_feature_importance,
    create_comparison_table,
    plot_score_distributions,
    generate_link_prediction_report
)

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid' if 'seaborn-v0_8-darkgrid' in plt.style.available 
              else 'seaborn-darkgrid' if 'seaborn-darkgrid' in plt.style.available 
              else 'ggplot')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")


## 2. Load Train Graph and Test Edges

Load the training graph and test edges (positive and negative) that were created during preprocessing.


In [ ]:
# Load training graph
train_graph_path = "data/processed/splits/train_graph.pkl"
print(f"Loading training graph from: {train_graph_path}")

with open(train_graph_path, 'rb') as f:
    G_train = pickle.load(f)

print(f"\n✅ Training graph loaded!")
print(f"   Nodes: {G_train.number_of_nodes():,}")
print(f"   Edges: {G_train.number_of_edges():,}")
print(f"   Type: {'Undirected' if not G_train.is_directed() else 'Directed'}")

# Load test edges
pos_test_path = "data/processed/splits/positive_test_edges.pkl"
neg_test_path = "data/processed/splits/negative_test_edges.pkl"

print(f"\nLoading test edges...")
with open(pos_test_path, 'rb') as f:
    pos_test_edges = pickle.load(f)

with open(neg_test_path, 'rb') as f:
    neg_test_edges = pickle.load(f)

print(f"✅ Test edges loaded!")
print(f"   Positive test edges: {len(pos_test_edges):,}")
print(f"   Negative test edges: {len(neg_test_edges):,}")
print(f"   Total test edges: {len(pos_test_edges) + len(neg_test_edges):,}")

# For faster computation, we'll use a sample of test edges
# You can adjust this based on your computational resources
SAMPLE_SIZE = min(5000, len(pos_test_edges))
print(f"\n📊 Using sample of {SAMPLE_SIZE} positive and {SAMPLE_SIZE} negative edges for evaluation")
pos_test_sample = pos_test_edges[:SAMPLE_SIZE]
neg_test_sample = neg_test_edges[:SAMPLE_SIZE]


## 3. Similarity-Based Link Prediction Methods

Implement and evaluate three similarity-based methods: Common Neighbors, Adamic-Adar, and Jaccard Coefficient.


### 3.1 Common Neighbors

Common Neighbors counts the number of shared neighbors between two nodes. Higher counts indicate higher likelihood of a link.


In [ ]:
print("Evaluating Common Neighbors method...")
print("=" * 60)

start_time = time.time()
cn_metrics = evaluate_link_prediction(
    G_train, 
    pos_test_sample, 
    neg_test_sample, 
    method='common_neighbors'
)
cn_time = time.time() - start_time

print(f"\n✅ Common Neighbors evaluation complete!")
print(f"   Execution time: {cn_time:.2f} seconds")
print(f"   Precision: {cn_metrics['precision']:.4f}")
print(f"   Recall: {cn_metrics['recall']:.4f}")
print(f"   F1 Score: {cn_metrics['f1']:.4f}")
print(f"   AUC-ROC: {cn_metrics['auc_roc']:.4f}")
print(f"   AUC-PR: {cn_metrics['auc_pr']:.4f}")
print(f"   Optimal Threshold: {cn_metrics['optimal_threshold']:.4f}")


### 3.2 Adamic-Adar

Adamic-Adar weights common neighbors by the inverse logarithm of their degree. Nodes with fewer neighbors contribute more to the score.


In [ ]:
print("Evaluating Adamic-Adar method...")
print("=" * 60)

start_time = time.time()
aa_metrics = evaluate_link_prediction(
    G_train, 
    pos_test_sample, 
    neg_test_sample, 
    method='adamic_adar'
)
aa_time = time.time() - start_time

print(f"\n✅ Adamic-Adar evaluation complete!")
print(f"   Execution time: {aa_time:.2f} seconds")
print(f"   Precision: {aa_metrics['precision']:.4f}")
print(f"   Recall: {aa_metrics['recall']:.4f}")
print(f"   F1 Score: {aa_metrics['f1']:.4f}")
print(f"   AUC-ROC: {aa_metrics['auc_roc']:.4f}")
print(f"   AUC-PR: {aa_metrics['auc_pr']:.4f}")
print(f"   Optimal Threshold: {aa_metrics['optimal_threshold']:.4f}")


### 3.3 Jaccard Coefficient

Jaccard Coefficient is the ratio of common neighbors to total unique neighbors. Range: [0, 1] where 1 indicates all neighbors are shared.


In [ ]:
print("Evaluating Jaccard Coefficient method...")
print("=" * 60)

start_time = time.time()
jc_metrics = evaluate_link_prediction(
    G_train, 
    pos_test_sample, 
    neg_test_sample, 
    method='jaccard'
)
jc_time = time.time() - start_time

print(f"\n✅ Jaccard Coefficient evaluation complete!")
print(f"   Execution time: {jc_time:.2f} seconds")
print(f"   Precision: {jc_metrics['precision']:.4f}")
print(f"   Recall: {jc_metrics['recall']:.4f}")
print(f"   F1 Score: {jc_metrics['f1']:.4f}")
print(f"   AUC-ROC: {jc_metrics['auc_roc']:.4f}")
print(f"   AUC-PR: {jc_metrics['auc_pr']:.4f}")
print(f"   Optimal Threshold: {jc_metrics['optimal_threshold']:.4f}")


### 3.4 Top Predictions for Similarity Methods

Show top predictions from each similarity method.


In [ ]:
# Get top predictions for each method
print("Generating top predictions...")

# Common Neighbors - top 10
cn_predictions = predict_links_similarity(G_train, method='common_neighbors', top_k=10)
print("\n📊 Top 10 Common Neighbors Predictions:")
print(cn_predictions.to_string(index=False))

# Adamic-Adar - top 10
aa_predictions = predict_links_similarity(G_train, method='adamic_adar', top_k=10)
print("\n📊 Top 10 Adamic-Adar Predictions:")
print(aa_predictions.to_string(index=False))

# Jaccard - top 10
jc_predictions = predict_links_similarity(G_train, method='jaccard', top_k=10)
print("\n📊 Top 10 Jaccard Coefficient Predictions:")
print(jc_predictions.to_string(index=False))


## 4. Machine Learning-Based Link Prediction

Implement Random Forest classifier using multiple features extracted from network structure.


### 4.1 Prepare Training Data

Extract features for positive and negative edges to create training data.


In [ ]:
# Use a subset for training (to balance speed and performance)
TRAIN_SIZE = min(2000, len(pos_test_sample))
print(f"Preparing training data with {TRAIN_SIZE} positive and {TRAIN_SIZE} negative edges...")
print("=" * 60)

start_time = time.time()
X_train, y_train = prepare_training_data(
    G_train, 
    pos_test_sample[:TRAIN_SIZE], 
    neg_test_sample[:TRAIN_SIZE]
)
prep_time = time.time() - start_time

print(f"\n✅ Training data prepared!")
print(f"   Feature extraction time: {prep_time:.2f} seconds")
print(f"   Training samples: {X_train.shape[0]:,}")
print(f"   Features: {X_train.shape[1]}")
print(f"   Positive samples: {np.sum(y_train == 1):,}")
print(f"   Negative samples: {np.sum(y_train == 0):,}")


### 4.2 Train Random Forest Model

Train a Random Forest classifier on the extracted features.


In [ ]:
print("Training Random Forest model...")
print("=" * 60)

start_time = time.time()
ml_model, scaler = train_random_forest(
    X_train, 
    y_train, 
    n_estimators=100,
    random_state=42,
    scale_features=True
)
ml_train_time = time.time() - start_time

print(f"\n✅ Model trained successfully!")
print(f"   Training time: {ml_train_time:.2f} seconds")
print(f"   Model type: Random Forest")
print(f"   Number of trees: 100")
print(f"   Features scaled: Yes")


### 4.3 Evaluate ML Model

Evaluate the Random Forest model on test data.


In [ ]:
# Use a subset for evaluation
EVAL_SIZE = min(1000, len(pos_test_sample))
print(f"Evaluating ML model on {EVAL_SIZE} positive and {EVAL_SIZE} negative test edges...")
print("=" * 60)

start_time = time.time()
ml_metrics, ml_predictions = evaluate_ml_model(
    ml_model,
    G_train,
    pos_test_sample[:EVAL_SIZE],
    neg_test_sample[:EVAL_SIZE],
    scaler=scaler
)
ml_eval_time = time.time() - start_time

print(f"\n✅ ML model evaluation complete!")
print(f"   Evaluation time: {ml_eval_time:.2f} seconds")
print(f"   Precision: {ml_metrics['precision']:.4f}")
print(f"   Recall: {ml_metrics['recall']:.4f}")
print(f"   F1 Score: {ml_metrics['f1']:.4f}")
print(f"   AUC-ROC: {ml_metrics['auc_roc']:.4f}")
print(f"   AUC-PR: {ml_metrics['auc_pr']:.4f}")
print(f"   Accuracy: {ml_metrics['accuracy']:.4f}")
print(f"\n   Confusion Matrix:")
print(f"   True Negatives: {ml_metrics['true_negatives']}")
print(f"   False Positives: {ml_metrics['false_positives']}")
print(f"   False Negatives: {ml_metrics['false_negatives']}")
print(f"   True Positives: {ml_metrics['true_positives']}")


### 4.4 Feature Importance

Analyze which features are most important for link prediction.


In [ ]:
# Get feature names
feature_names = ['common_neighbors', 'jaccard_coefficient', 'adamic_adar',
                'degree_u', 'degree_v', 'degree_product', 'degree_sum',
                'clustering_u', 'clustering_v']

# Extract feature importance
importance_df = get_feature_importance(ml_model, feature_names)

print("📊 Feature Importances (Random Forest):")
print("=" * 60)
print(importance_df.to_string(index=False))

# Plot feature importance
os.makedirs("results/figures", exist_ok=True)
plot_feature_importance(
    importance_df,
    save_path="results/figures/link_prediction_feature_importance.png"
)
print("\n✅ Feature importance plot saved: results/figures/link_prediction_feature_importance.png")


## 5. Comprehensive Comparison of All Methods

Compare all four methods (3 similarity-based + 1 ML-based) using various metrics and visualizations.


### 5.1 Comparison Table

Create a comprehensive comparison table with all metrics.


In [ ]:
# Prepare all results
all_results = {
    'Common Neighbors': cn_metrics,
    'Adamic-Adar': aa_metrics,
    'Jaccard Coefficient': jc_metrics,
    'Random Forest (ML)': ml_metrics
}

# Create comparison table
comparison_df, styled_df = create_comparison_table(all_results)

print("📊 Comparison of All Link Prediction Methods:")
print("=" * 80)
print(comparison_df.to_string(index=False))
print("=" * 80)


### 5.2 ROC Curves Comparison

Plot ROC curves for all methods on the same plot.


In [ ]:
# Calculate ROC curves for all methods
from sklearn.metrics import roc_curve

print("Calculating ROC curves for all methods...")

roc_data = {}

# Common Neighbors
cn_scores_pos = common_neighbors_score(G_train, pos_test_sample[:EVAL_SIZE])
cn_scores_neg = common_neighbors_score(G_train, neg_test_sample[:EVAL_SIZE])
cn_all_scores = np.array(cn_scores_pos + cn_scores_neg)
cn_all_labels = np.array([1] * len(cn_scores_pos) + [0] * len(cn_scores_neg))
cn_fpr, cn_tpr, _ = roc_curve(cn_all_labels, cn_all_scores)
roc_data['Common Neighbors'] = (cn_fpr, cn_tpr, cn_metrics['auc_roc'])

# Adamic-Adar
aa_scores_pos = adamic_adar_score(G_train, pos_test_sample[:EVAL_SIZE])
aa_scores_neg = adamic_adar_score(G_train, neg_test_sample[:EVAL_SIZE])
aa_all_scores = np.array(aa_scores_pos + aa_scores_neg)
aa_all_labels = np.array([1] * len(aa_scores_pos) + [0] * len(aa_scores_neg))
aa_fpr, aa_tpr, _ = roc_curve(aa_all_labels, aa_all_scores)
roc_data['Adamic-Adar'] = (aa_fpr, aa_tpr, aa_metrics['auc_roc'])

# Jaccard
jc_scores_pos = jaccard_coefficient_score(G_train, pos_test_sample[:EVAL_SIZE])
jc_scores_neg = jaccard_coefficient_score(G_train, neg_test_sample[:EVAL_SIZE])
jc_all_scores = np.array(jc_scores_pos + jc_scores_neg)
jc_all_labels = np.array([1] * len(jc_scores_pos) + [0] * len(jc_scores_neg))
jc_fpr, jc_tpr, _ = roc_curve(jc_all_labels, jc_all_scores)
roc_data['Jaccard Coefficient'] = (jc_fpr, jc_tpr, jc_metrics['auc_roc'])

# Random Forest
ml_probs = ml_predictions['probability'].values
ml_labels = np.array([1] * EVAL_SIZE + [0] * EVAL_SIZE)
ml_fpr, ml_tpr, _ = roc_curve(ml_labels, ml_probs)
roc_data['Random Forest (ML)'] = (ml_fpr, ml_tpr, ml_metrics['auc_roc'])

# Plot ROC curves
plot_roc_curves(roc_data, save_path="results/figures/link_prediction_roc_curves.png")
print("✅ ROC curves saved: results/figures/link_prediction_roc_curves.png")


### 5.3 Precision-Recall Curves Comparison

Plot Precision-Recall curves for all methods.


In [ ]:
# Calculate PR curves for all methods
from sklearn.metrics import precision_recall_curve

print("Calculating Precision-Recall curves for all methods...")

pr_data = {}

# Common Neighbors
cn_precision, cn_recall, _ = precision_recall_curve(cn_all_labels, cn_all_scores)
pr_data['Common Neighbors'] = (cn_precision, cn_recall, cn_metrics['auc_pr'])

# Adamic-Adar
aa_precision, aa_recall, _ = precision_recall_curve(aa_all_labels, aa_all_scores)
pr_data['Adamic-Adar'] = (aa_precision, aa_recall, aa_metrics['auc_pr'])

# Jaccard
jc_precision, jc_recall, _ = precision_recall_curve(jc_all_labels, jc_all_scores)
pr_data['Jaccard Coefficient'] = (jc_precision, jc_recall, jc_metrics['auc_pr'])

# Random Forest
ml_precision, ml_recall, _ = precision_recall_curve(ml_labels, ml_probs)
pr_data['Random Forest (ML)'] = (ml_precision, ml_recall, ml_metrics['auc_pr'])

# Plot PR curves
plot_precision_recall_curves(pr_data, save_path="results/figures/link_prediction_pr_curves.png")
print("✅ Precision-Recall curves saved: results/figures/link_prediction_pr_curves.png")


### 5.4 Confusion Matrices

Visualize confusion matrices for all methods.


In [ ]:
# Calculate confusion matrices
from sklearn.metrics import confusion_matrix

# For similarity methods, use optimal thresholds
cn_pred = (cn_all_scores >= cn_metrics['optimal_threshold']).astype(int)
aa_pred = (aa_all_scores >= aa_metrics['optimal_threshold']).astype(int)
jc_pred = (jc_all_scores >= jc_metrics['optimal_threshold']).astype(int)

# For ML, use predictions
ml_pred = ml_predictions['prediction'].values

confusion_matrices = {
    'Common Neighbors': confusion_matrix(cn_all_labels, cn_pred),
    'Adamic-Adar': confusion_matrix(aa_all_labels, aa_pred),
    'Jaccard Coefficient': confusion_matrix(jc_all_labels, jc_pred),
    'Random Forest (ML)': confusion_matrix(ml_labels, ml_pred)
}

# Plot confusion matrices
plot_confusion_matrices(confusion_matrices, save_path="results/figures/link_prediction_confusion_matrices.png")
print("✅ Confusion matrices saved: results/figures/link_prediction_confusion_matrices.png")


### 5.5 Score Distributions

Compare score distributions for positive and negative edges.


In [ ]:
# Plot score distributions for each method
score_distributions = {
    'Common Neighbors': (np.array(cn_scores_pos), np.array(cn_scores_neg)),
    'Adamic-Adar': (np.array(aa_scores_pos), np.array(aa_scores_neg)),
    'Jaccard Coefficient': (np.array(jc_scores_pos), np.array(jc_scores_neg)),
    'Random Forest (ML)': (
        ml_predictions[ml_predictions['prediction'] == 1]['probability'].values[:EVAL_SIZE],
        ml_predictions[ml_predictions['prediction'] == 0]['probability'].values[:EVAL_SIZE]
    )
}

for method_name, (pos_scores, neg_scores) in score_distributions.items():
    plot_score_distributions(
        pos_scores,
        neg_scores,
        method_name,
        save_path=f"results/figures/link_prediction_score_dist_{method_name.replace(' ', '_').replace('(', '').replace(')', '')}.png"
    )

print("✅ Score distribution plots saved")


## 6. Results Analysis and Interpretation

Analyze the results and provide insights about which method works best and why.


### 6.1 Performance Summary

**Which Method Works Best?**

Based on the comparison table above:

1. **Best F1 Score**: [Method with highest F1]
2. **Best AUC-ROC**: [Method with highest AUC-ROC]
3. **Best AUC-PR**: [Method with highest AUC-PR]
4. **Best Precision**: [Method with highest Precision]
5. **Best Recall**: [Method with highest Recall]

**Key Observations:**
- Random Forest (ML) typically performs best because it combines multiple features
- Similarity-based methods are faster but may have lower accuracy
- Adamic-Adar often outperforms Common Neighbors and Jaccard
- The choice of method depends on the trade-off between speed and accuracy


### 6.2 Feature Importance Analysis

**What Features Are Most Important?**

From the feature importance plot above, we can see:

1. **Top Features**: [List top 3-5 features]
2. **Insights**:
   - Similarity-based features (Adamic-Adar, Common Neighbors, Jaccard) are typically most important
   - Node degree features provide additional signal
   - Clustering coefficients may have lower importance but still contribute

**Interpretation:**
- Features that capture local network structure are most predictive
- Combining multiple features improves prediction accuracy
- Some features may be redundant (e.g., degree_product and degree_sum)


### 6.3 Example Predictions

Show some example predictions from the best-performing method.


In [ ]:
# Get top predictions from best method (Random Forest)
print("📊 Top 20 Predictions from Random Forest (ML):")
print("=" * 80)
top_ml_predictions = ml_predictions.head(20)
print(top_ml_predictions.to_string(index=False))

# Show some examples of high-confidence predictions
print("\n📊 High-Confidence Predictions (Probability > 0.9):")
print("=" * 80)
high_conf = ml_predictions[ml_predictions['probability'] > 0.9].head(10)
print(high_conf.to_string(index=False))

# Show some examples of low-confidence predictions
print("\n📊 Low-Confidence Predictions (Probability < 0.1):")
print("=" * 80)
low_conf = ml_predictions[ml_predictions['probability'] < 0.1].head(10)
print(low_conf.to_string(index=False))


### 6.4 Trade-offs: Speed vs Accuracy

**Execution Time Comparison:**

| Method | Training Time | Evaluation Time | Total Time |
|--------|--------------|-----------------|------------|
| Common Neighbors | N/A | [time] | [time] |
| Adamic-Adar | N/A | [time] | [time] |
| Jaccard Coefficient | N/A | [time] | [time] |
| Random Forest (ML) | [time] | [time] | [time] |

**Trade-offs:**

1. **Similarity-Based Methods:**
   - ✅ **Pros**: Very fast, no training required, interpretable
   - ❌ **Cons**: Lower accuracy, single feature, may not capture complex patterns

2. **ML-Based Methods:**
   - ✅ **Pros**: Higher accuracy, combines multiple features, learns complex patterns
   - ❌ **Cons**: Slower (requires training), more complex, less interpretable

**Recommendations:**
- Use **similarity-based methods** for:
  - Real-time predictions
  - Large-scale networks where speed is critical
  - When interpretability is important
  
- Use **ML-based methods** for:
  - When accuracy is the priority
  - When you have computational resources
  - When you can pre-train models offline


### 6.5 Business Insights

**Applications in E-commerce:**

1. **Product Recommendations:**
   - Predict which products customers are likely to co-purchase
   - Improve recommendation systems
   - Increase cross-selling opportunities

2. **Inventory Management:**
   - Identify product bundles that are frequently purchased together
   - Optimize warehouse layout
   - Improve supply chain efficiency

3. **Marketing Strategies:**
   - Target customers with complementary products
   - Design promotional bundles
   - Optimize advertising campaigns

4. **Network Growth:**
   - Predict which new products will connect to existing network
   - Identify potential partnerships
   - Understand market dynamics

**Key Metrics for Business:**
- **Precision**: Percentage of predicted links that are actually true (important for avoiding false recommendations)
- **Recall**: Percentage of actual links that are predicted (important for not missing opportunities)
- **F1 Score**: Balanced measure of precision and recall
- **AUC-ROC**: Overall ranking quality of predictions


## 7. Save All Results

Save all results, metrics, and visualizations to the results directory.


In [ ]:
# Create output directory
output_dir = "data/results/link_prediction"
os.makedirs(output_dir, exist_ok=True)
os.makedirs("results/tables", exist_ok=True)

print(f"Saving all results to {output_dir}/...")

# Save comparison table
comparison_df.to_csv(os.path.join(output_dir, "comparison_table.csv"), index=False)
print(f"✅ Comparison table saved: {output_dir}/comparison_table.csv")

# Save individual method metrics
metrics_dict = {
    'common_neighbors': cn_metrics,
    'adamic_adar': aa_metrics,
    'jaccard_coefficient': jc_metrics,
    'random_forest': ml_metrics
}

import json
with open(os.path.join(output_dir, "all_metrics.json"), 'w') as f:
    json.dump(metrics_dict, f, indent=2)
print(f"✅ All metrics saved: {output_dir}/all_metrics.json")

# Save top predictions
cn_predictions.to_csv(os.path.join(output_dir, "top_predictions_common_neighbors.csv"), index=False)
aa_predictions.to_csv(os.path.join(output_dir, "top_predictions_adamic_adar.csv"), index=False)
jc_predictions.to_csv(os.path.join(output_dir, "top_predictions_jaccard.csv"), index=False)
ml_predictions.head(100).to_csv(os.path.join(output_dir, "top_predictions_random_forest.csv"), index=False)
print(f"✅ Top predictions saved: {output_dir}/top_predictions_*.csv")

# Save feature importance
importance_df.to_csv(os.path.join(output_dir, "feature_importance.csv"), index=False)
print(f"✅ Feature importance saved: {output_dir}/feature_importance.csv")

# Generate comprehensive HTML report
print("\nGenerating comprehensive HTML report...")
generate_link_prediction_report(
    all_results,
    'results/tables/link_prediction_report.html',
    roc_data=roc_data,
    pr_data=pr_data,
    confusion_matrices=confusion_matrices,
    feature_importance=importance_df,
    score_distributions=score_distributions
)
print("✅ HTML report created: results/tables/link_prediction_report.html")

print(f"\n✅ All results saved successfully!")
print(f"   Output directory: {output_dir}/")
print(f"   Figures: results/figures/")
print(f"   Reports: results/tables/")


## Summary

This notebook has completed comprehensive link prediction analysis:

✅ **Data Loaded**: Training graph and test edges (positive and negative)  
✅ **Similarity Methods**: Common Neighbors, Adamic-Adar, Jaccard Coefficient evaluated  
✅ **ML Method**: Random Forest trained and evaluated with 9 features  
✅ **Comparison**: All 4 methods compared with metrics and visualizations  
✅ **Analysis**: Performance analysis, feature importance, and business insights  
✅ **Results Saved**: All outputs saved to data/results/link_prediction/  

### Key Findings:
- [Best performing method] achieves highest accuracy
- [Top features] are most important for prediction
- Trade-offs between speed and accuracy identified
- Business applications and recommendations provided

### Next Steps:
- Experiment with different ML models (XGBoost, Neural Networks)
- Add more features (temporal, content-based)
- Evaluate on different network datasets
- Deploy best model for production use
